In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd


In [ ]:
def load_data(test_case_path):
    """Loads images and ground truth data for a given test case."""
    search_area_path = os.path.join(test_case_path, 'search_area_mask.png')
    search_area = cv2.imread(search_area_path, cv2.IMREAD_GRAYSCALE)
    if search_area is None:
        raise FileNotFoundError(f"Could not read search area image at: {search_area_path}")
        
    results_df = pd.read_csv(os.path.join(test_case_path, 'result.csv'))
    
    templates = []
    template_dir = os.path.join(test_case_path, 'regoins_segments')
    for i in range(len(results_df)):
        template_path = os.path.join(template_dir, f'{i}.png')
        template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
        if template is None:
            print(f"Warning: Could not read template image at: {template_path}")
            continue
        templates.append(template)
        
    return search_area, templates, results_df

In [ ]:
def multi_scale_template_matching(search_area, template):
    """
    Performs multi-scale template matching, returning the best match location, size, and score.
    """
    best_match = None
    max_val = -np.inf
    
    # Iterate through a range of scales
    for scale in np.linspace(0.5, 1.5, 50)[::-1]:
        resized_template = cv2.resize(template, (0, 0), fx=scale, fy=scale)
        if resized_template.shape[0] > search_area.shape[0] or resized_template.shape[1] > search_area.shape[1]:
            continue
        
        # Perform template matching
        result = cv2.matchTemplate(search_area, resized_template, cv2.TM_CCOEFF_NORMED)
        _, current_max_val, _, max_loc = cv2.minMaxLoc(result)
        
        # Keep track of the best match (highest score)
        if current_max_val > max_val:
            max_val = current_max_val
            best_match = (max_loc, resized_template.shape[::-1], max_val) # (top_left, (w, h), score)
            
    return best_match

In [ ]:
def calculate_error(predicted_tl, ground_truth_tl):
    """Calculates the Euclidean distance between two points."""
    return np.sqrt((predicted_tl[0] - ground_truth_tl[0])**2 + (predicted_tl[1] - ground_truth_tl[1])**2)


In [ ]:
test_1_path = 'matching/tests/1/'
test_2_rotated_path = 'matching/tests/2_rotated/'
all_test_cases = [test_1_path, test_2_rotated_path]

In [ ]:
for test_path in all_test_cases:
        case_name = os.path.basename(os.path.normpath(test_path))
        print(f"\n{'='*60}\nEVALUATING: {case_name} with Multi-Scale Template Matching\n{'='*60}")
        
        
        search_area, templates, results_df = load_data(test_path)

        errors = []
        
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.title('Search Area')
        plt.imshow(search_area, cmap='gray')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.title('Example Template (0.png)')
        plt.imshow(templates[0], cmap='gray')
        plt.axis('off')
        plt.suptitle(f'Inputs for {case_name}', fontsize=16)
        plt.show()

        # --- Loop through each template, match, evaluate, and visualize ---
        # Set to True to see plots for every single image, False to just see the first one.
        VISUALIZE_ALL_STEPS = True 
        
        for i, template in enumerate(templates):
            print(f"\n--- Processing Template {i} ---")
            
            # Find the best match
            match = multi_scale_template_matching(search_area, template)

            predicted_tl, size, score = match
            
            # Get ground truth for comparison
            gt_row = results_df.iloc[i]
            gt_tl = (gt_row['top_left_x'], gt_row['top_left_y'])
            gt_br = (gt_row['bottom_right_x'], gt_row['bottom_right_y'])
            
            # Calculate error
            error = calculate_error(predicted_tl, gt_tl)
            errors.append(error)
            
            print(f"Match Score: {score:.4f}")
            print(f"Predicted Top-Left: {predicted_tl}")
            print(f"Ground Truth Top-Left: {gt_tl}")
            print(f"Error: {error:.2f} pixels")
            
            # --- Visualize the result for this specific template ---
            if VISUALIZE_ALL_STEPS or i == 0:
                # Create a color image to draw colored boxes
                vis_image = cv2.cvtColor(search_area, cv2.COLOR_GRAY2BGR)
                
                # Draw predicted bounding box (Green)
                predicted_br = (predicted_tl[0] + size[0], predicted_tl[1] + size[1])
                cv2.rectangle(vis_image, predicted_tl, predicted_br, (0, 255, 0), 2)
                cv2.putText(vis_image, 'Predicted', (predicted_tl[0], predicted_tl[1] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                # Draw ground truth bounding box (Red)
                cv2.rectangle(vis_image, gt_tl, gt_br, (0, 0, 255), 2)
                cv2.putText(vis_image, 'Ground Truth', (gt_tl[0], gt_tl[1] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                plt.figure(figsize=(10, 8))
                plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
                plt.title(f'{case_name} - Template {i}\nError: {error:.2f} pixels')
                plt.axis('off')
                plt.show()

        if errors:
            avg_error = np.mean(errors)
            all_errors[case_name] = avg_error
            print(f"\nAverage Error for {case_name}: {avg_error:.2f} pixels")
        else:
            all_errors[case_name] = np.inf

    # --- Final Summary Visualization ---
plt.figure(figsize=(8, 6))
plt.bar(all_errors.keys(), all_errors.values(), color=['skyblue', 'salmon'])
plt.ylabel('Average Pixel Error')
plt.title('Template Matching Performance')
for i, (k, v) in enumerate(all_errors.items()):
    plt.text(i, v + 0.5, f'{v:.2f}', ha='center')
plt.show()

## Evaluation: Compare Detected Region to Ground Truth Center

This section compares the detected region's center to the ground truth center from result.csv and visualizes both.

In [ ]:
# Load ground truth center from result.csv
result_path = os.path.join(TEST_CASE_DIR, 'result.csv')
df = pd.read_csv(result_path)
template_id = int(TEMPLATE_NAME.split('.')[0])
row = df[df['Test ID'] == template_id].iloc[0]
gt_center = np.array([row['Main X'], row['Main Y']])

In [ ]:
# Compute detected center from transformed corners
detected_center = np.mean(transformed_corners.squeeze(), axis=0)
center_dist = np.linalg.norm(detected_center - gt_center)
print(f"Detected center: {detected_center.astype(int)}")
print(f"Ground truth center: {gt_center}")
print(f"Center distance: {center_dist:.2f} pixels")

In [ ]:
# Visualize detected region and ground truth center
img = search_img_bgr.copy()
cv2.polylines(img, [np.int32(transformed_corners)], True, (0,255,0), 3)
cv2.circle(img, tuple(gt_center.astype(int)), 12, (255,0,0), -1)
cv2.circle(img, tuple(detected_center.astype(int)), 12, (0,255,0), -1)
plt.figure(figsize=(10,8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Detected (green) vs Ground Truth Center (blue)')
plt.axis('off')
plt.show()